In [1]:
import torch
import numpy as np
from scenarionet import read_dataset_summary, read_scenario
from metadrive.engine.asset_loader import AssetLoader

In [2]:
torch.cuda.empty_cache()

In [4]:
av2_data =  AssetLoader.file_path("/home/light/Documents/Thesis/preprocessed_dataset", unix_style=False)
dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path=av2_data)

scenario_file_name = scenario_ids[0]
scenario = read_scenario(dataset_path=av2_data, mapping=mapping, scenario_file_name=scenario_file_name)
scenario.keys()

dict_keys(['id', 'version', 'length', 'tracks', 'dynamic_map_states', 'map_features', 'metadata'])

In [12]:
import torch
import numpy as np

class AdaptiveTokenScenarioNet:
    def __init__(self, importance_threshold=0.5, max_tokens=512, token_length=10, use_gpu=False):
        self.importance_threshold = importance_threshold
        self.max_tokens = max_tokens
        self.token_length = token_length
        self.device = torch.device('cuda' if (use_gpu and torch.cuda.is_available()) else 'cpu')
        self.type_importance = {
            'VEHICLE': 1.0,
            'PEDESTRIAN': 0.8,
            'BICYCLE': 0.7,
            'STATIC': 0.5,
            'UNKNOWN': 0.3
        }
        self.map_feature_importance = {
            'LANE': 0.9,
            'CROSSWALK': 0.7,
            'STOP_SIGN': 0.6,
            'TRAFFIC_LIGHT': 0.8,
            'UNKNOWN': 0.4
        }

    def score_tracks(self, tracks):
        importance_scores = {}
        for track_id, track in tracks.items():
            state = track['state']
            valid = state['valid']
            if not valid.any():
                importance_scores[track_id] = 0
                continue

            valid_positions = state['position'][valid]
            valid_velocities = state['velocity'][valid]
            valid_headings = state['heading'][valid]
            valid_lengths = state['length'][valid]
            valid_widths = state['width'][valid]
            valid_heights = state['height'][valid]

            activity_score = np.linalg.norm(valid_positions, axis=1).sum()
            velocity_score = np.linalg.norm(valid_velocities, axis=1).sum()
            heading_score = np.abs(valid_headings).sum()
            size_score = valid_lengths.sum() + valid_widths.sum() + valid_heights.sum()
            type_score = self.type_importance.get(track.get('type', 'UNKNOWN'), 0.3)

            importance_scores[track_id] = activity_score + velocity_score + heading_score + size_score + type_score
        return importance_scores

    def score_map_features(self, map_features):
        importance_scores = {}
        for feature_id, feature in map_features.items():
            feature_type = feature.get('type', 'UNKNOWN')
            polyline = feature.get('polyline', [])
            if len(polyline) == 0:
                importance_scores[feature_id] = 0
                continue

            feature_score = len(polyline) * self.map_feature_importance.get(feature_type, 0.4)
            importance_scores[feature_id] = feature_score
        return importance_scores

    def pad_or_truncate(self, token, expected_dim):
        if token.shape[1] != expected_dim:
            token = token[:, :expected_dim] if token.shape[1] > expected_dim else np.pad(
                token, ((0, 0), (0, expected_dim - token.shape[1])), mode='constant'
            )
        if token.shape[0] > self.token_length:
            token = token[:self.token_length]
        else:
            pad_length = self.token_length - token.shape[0]
            pad = np.zeros((pad_length, expected_dim), dtype=np.float32)
            token = np.vstack((token, pad))
        return token

    def tokenize_tracks(self, tracks):
        tokenized_tracks = {}
        for track_id, track in tracks.items():
            state = track['state']
            valid = state['valid']
            if not valid.any():
                tokenized_tracks[track_id] = torch.empty((self.token_length, 9), dtype=torch.float32, device=self.device)
                continue

            valid_positions = state['position'][valid]
            valid_velocities = state['velocity'][valid]
            valid_headings = state['heading'][valid]
            valid_lengths = state['length'][valid]
            valid_widths = state['width'][valid]
            valid_heights = state['height'][valid]

            combined_token = np.column_stack((valid_positions, valid_velocities, valid_headings, valid_lengths, valid_widths, valid_heights))
            token = self.pad_or_truncate(combined_token, expected_dim=9)
            tokenized_tracks[track_id] = torch.tensor(token, dtype=torch.float32, device=self.device)
        return tokenized_tracks

    def tokenize_map_features(self, map_features):
        tokenized_features = {}
        for feature_id, feature in map_features.items():
            polyline = feature.get('polyline', [])
            if len(polyline) == 0:
                tokenized_features[feature_id] = torch.empty((self.token_length, 2), dtype=torch.float32, device=self.device)
                continue

            polyline_array = np.array(polyline)
            polyline_array = polyline_array[:, :2] if polyline_array.shape[1] > 2 else polyline_array
            token = self.pad_or_truncate(polyline_array, expected_dim=2)
            tokenized_features[feature_id] = torch.tensor(token, dtype=torch.float32, device=self.device)
        return tokenized_features

    def process(self, scenario):
        tracks = scenario.get('tracks', {})
        map_features = scenario.get('map_features', {})
        metadata = scenario.get('metadata', {})

        track_scores = self.score_tracks(tracks)
        map_scores = self.score_map_features(map_features)

        tokenized_tracks = self.tokenize_tracks(tracks)
        tokenized_map_features = self.tokenize_map_features(map_features)

        scenario['tracks'] = {
            track_id: {
                **track,
                'importance_score': track_scores[track_id],
                'token': tokenized_tracks[track_id]
            } for track_id, track in tracks.items()
        }

        scenario['map_features'] = {
            feature_id: {
                **feature,
                'importance_score': map_scores[feature_id],
                'token': tokenized_map_features[feature_id]
            } for feature_id, feature in map_features.items()
        }
        
        return scenario


In [11]:
tokenizer = AdaptiveTokenScenarioNet(token_length=10, use_gpu=True)
tokens = tokenizer.process(scenario)

print("Tokenization Complete!")
print(f"Tracks Processed: {len(tokens['tracks'])}")
print(f"Map Features Processed: {len(tokens['map_features'])}")
print(f"Metadata: {tokens['metadata']}")


Tokenization Complete!
Tracks Processed: 21
Map Features Processed: 212
Metadata: {'id': '0a14e451-2293-4a95-ab65-184427d890f3', 'coordinate': 'waymo', 'ts': array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
        1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,
        2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,
        4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,
        5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,
        6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,
        7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,
        8.8,  8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,
        9.9, 10. , 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9]), 'metadrive_processed': False, 'sdc_id': 'AV', 'dataset': 'av2', 'scenario_id': '0a14e451-2293-4a95-ab65-184427

In [17]:
tokens['tracks']['73980']['importance_score']

np.float32(345847.72)